# Table Tutorial

## Preprocessing

In [ ]:
import os
os.remove('snorkel.db')

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Parse the `Corpus`

In [ ]:
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

doc_parser = HTMLParser(path='data/hardware/hardware_html/')
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser, max_docs=101)

In [ ]:
%time corpus = cp.parse_corpus(name='Hardware Training', session=session)

In [ ]:
for doc in corpus.documents[:5]: print doc

### Saving the `Corpus`
Finally, we persist the parsed corpus in Snorkel's database backend:

In [ ]:
session.add(corpus)
session.commit()

### Reloading the `Corpus`
If the corpus has already been parsed, load it here:

In [2]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Training').one()
print corpus
print "%s contains %d Documents" % (corpus.name, len(corpus))

Corpus (Hardware Training)
Hardware Training contains 100 Documents


## Extract the `Candidates`

In [3]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

In [ ]:
from snorkel.candidates import OmniNgrams

omni_ngrams = OmniNgrams(n_max=3, split_tokens=None)

In [ ]:
from hardware_utils import load_extended_parts_dict

filename='data/hardware/gold_all.csv'
parts_dict = load_extended_parts_dict(filename)
print "Loaded %d part numbers." % len(parts_dict)

In [ ]:
from snorkel.matchers import DictionaryMatch, RegexMatchSpan

parts_matcher = DictionaryMatch(d=parts_dict, ignore_case=True)
temp_matcher = RegexMatchSpan(rgx=ur'-\s?[5-7][05]')

In [ ]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part_Temp, [omni_ngrams, omni_ngrams], [parts_matcher, temp_matcher])

In [ ]:
%time candidates = ce.extract(corpus.documents, 'Hardware Training Candidates', session)
print "Number of candidates:", len(candidates)

In [ ]:
# session.rollback()
# session.delete(c)
# session.commit()

In [ ]:
for c in candidates[:5]:
    print c

### Saving the extracted candidates

In [ ]:
session.add(candidates)
session.commit()

### Reloading the candidates

In [4]:
from snorkel.models import CandidateSet

candidates = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
print candidates
print "%s contains %d Candidates" % (candidates.name, len(candidates))

Candidate Set (Hardware Training Candidates)
Hardware Training Candidates contains 646 Candidates


## Prepare Gold Data

In [ ]:
# Make labels gold, candidates gold

In [30]:
# TEMP
import os
os.remove('snorkel.db')
os.system('cp snorkel.db\ candidates snorkel.db')
from snorkel import SnorkelSession
session = SnorkelSession()
from snorkel.models import Corpus
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Training').one()
print corpus
print "%s contains %d Documents" % (corpus.name, len(corpus))
from snorkel.models import CandidateSet
candidates = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
print candidates
print "%s contains %d Candidates" % (candidates.name, len(candidates))

Corpus (Hardware Training)
Hardware Training contains 100 Documents
Candidate Set (Hardware Training Candidates)
Hardware Training Candidates contains 646 Candidates


In [31]:
from snorkel.loaders import ExternalAnnotationsLoader

loader = ExternalAnnotationsLoader(session, Part_Temp, 
                                   'Hardware Development Candidates -- Gold',
                                   'Hardware Development Labels -- Gold',
                                   expand_candidate_set=True)

In [32]:
import os
from hardware_utils import load_hardware_labels

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
load_hardware_labels(loader, candidates, filename, 'Part_Temp', attrib='stg_temp_min')

In [17]:
from snorkel.models import Label

dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates -- Gold').one()
print len(dev)
print session.query(Label).filter(Label.key == loader.annotation_key).count()

0
153


In [10]:
print dev

Candidate Set (Hardware Development Candidates -- Gold)


In [ ]:
# for set_name in ['Training', 'Test']:
#     loader = ExternalAnnotationsLoader(session, Disease, 
#                                        'CDR %s Candidates -- Gold' % set_name,
#                                        'CDR %s Labels -- Gold' % set_name,
#                                        expand_candidate_set=True)
#     load_BioC_disease_labels(loader, 'CDR_%sSet.BioC.xml' % set_name)
#     print session.query(Label).filter(Label.key == loader.annotation_key).count()

## Generating Features

In [ ]:
from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(CandidateSet.name == 'Hardware Training Candidates').one()

In [ ]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

### Creating feature matrix

In [ ]:
%time F_train = feature_manager.create(session, train, 'Train Features')

### Reloading feature matrix

In [ ]:
%time F_train = feature_manager.load(session, train, 'Train Features')

In [ ]:
F_train

In [ ]:
F_train.get_candidate(0)

In [ ]:
F_train.get_key(0)

In [ ]:
from snorkel.utils import get_keys_by_candidate

for f in get_keys_by_candidate(F_train, F_train.get_candidate(0)): print f

## Applying LFs

In [ ]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

In [ ]:
# TEMP
# from snorkel import SnorkelSession
# session = SnorkelSession()
# from snorkel.models import candidate_subclass
# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])
# from snorkel.models import CandidateSet
# candidates = session.query(CandidateSet).filter(CandidateSet.name == 'Hardware Training Candidates').one()
# print candidates
# train = candidates

In [ ]:
from snorkel.lf_helpers import get_right_ngrams, get_left_ngrams, contains_token
from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
cand = train[0]
print cand
print cand[1].parent.cell.text
print get_left_ngrams(cand[1])
print get_right_ngrams(cand[1])
print get_phrase_ngrams(cand[1])
print get_cell_ngrams(cand[1])
print get_neighbor_cell_ngrams(cand[1])
print get_row_ngrams(cand[1])
print get_col_ngrams(cand[1])
print get_aligned_ngrams(cand[1])

In [ ]:
from snorkel.lf_helpers import *

LFs = []

def LF_to_range(c):
    return 1 if 'to' in get_right_ngrams(c) else 0
LFs.append(LF_to_range)

def LF_tilde_range(c):
    return 1 if '~' in get_right_ngrams(c) else 0
LFs.append(LF_tilde_range)

def LF_through_range(c):
    return 1 if set(['through','thru']).intersection(set(get_right_ngrams(c))) > 0 else 0
LFs.append(LF_through_range)

def LF_contains_minus(c):
    return 1 if contains_token(c, '-') or contains_token(c,'-50') else -1
LFs.append(LF_contains_minus)

def LF_storage(c):
    return 1 if 'storage' in get_row_ngrams(c, attrib='words') else -1
LFs.append(LF_storage)

def LF_tstg(c):
    return 1 if 'tstg' in get_row_ngrams(c, attrib='words') else -1
LFs.append(LF_tstg)

def LF_tj(c):
    return 1 if 'tj' in get_row_ngrams(c, attrib='words') else -1
LFs.append(LF_tj)

def LF_temperature(c):
    return 1 if 'temperature' in get_row_ngrams(c, attrib='words') else -1
LFs.append(LF_temperature)

def LF_celsius(c):
    return 1 if 'c' in get_row_ngrams(c, attrib='words') else -1
LFs.append(LF_celsius)

def LF_max(c):
    return 1 if 'max' in get_aligned_ngrams(c, attrib='words') else 0
LFs.append(LF_max)

def LF_min(c):
    return 1 if 'min' in get_aligned_ngrams(c, attrib='words') else 0
LFs.append(LF_min)

In [ ]:
%time L_train = label_manager.create(session, train, 'LF Labels2', f=LFs)
L_train

In [ ]:
L_train.lf_stats()

## Train Generative Model

In [ ]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=3000, rate=1e-5)

In [ ]:
gen_model.save(session, 'Generative Params')

In [ ]:
train_marginals = gen_model.marginals(L_train)

## Train Discriminative Model

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=5000, rate=1e-3)

In [ ]:
disc_model.w.shape

In [ ]:
%time disc_model.save(session, "Discriminative Params")

## Assess Performance on Dev Set

In [ ]:
# Temporarily test on the training set:
from snorkel.models import CandidateSet
dev = session.query(CandidateSet).filter(CandidateSet.name == 'Hardware Training Candidates').one()

In [ ]:
%time F_dev = feature_manager.update(session, dev, 'Train Features', False)

In [ ]:
L_dev = label_manager.load(session, dev, "CDR Development Labels -- Gold")

In [ ]:
gold_dev_set = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Development Candidates -- Gold').one()

In [ ]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)